# TextExplainer: Debugging black box text classifier




`eli5.lime` can help when it is hard to get exact mapping between model coefficients and text features, e.g. if there are dimensionality reduction involved [^1].


[^1]: https://eli5.readthedocs.io/en/latest/tutorials/black-box-text-classifiers.html

## Example problem: LSA+SVM for 20 Newsgroups dataset

We will create a sample text processing pipeline which is hard to debug using conventional method: SVM with RBF kernel trained on LSA features.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]

twenty_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=["headers", "footers"],
)
twenty_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=["headers", "footers"],
)

In [2]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC

vec = TfidfVectorizer(min_df=3, stop_words="english", ngram_range=(1, 2))
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

clf = SVC(C=150, gamma=2e-2, probability=True)
pipe = make_pipeline(lsa, clf)
pipe.fit(twenty_train.data, twenty_train.target)
pipe.score(twenty_test.data, twenty_test.target)

0.8901464713715047

In [3]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(twenty_train.target_names, y_pred):
        print("{:.3f} {}".format(prob, target))


doc = twenty_test.data[0]
print_prediction(doc)

0.001 alt.atheism
0.001 comp.graphics
0.996 sci.med
0.003 soc.religion.christian


The classifier predicts that the first document belongs to `sci.med` with high probability.

## TextExplainer

eli5 does not support the pipeline directly, but we can use `TextExplainer`. 

We create an instance of `TextClassifier`, then pass the doc and the black-box classifier which implements `predict_proba` method to the `fit` method.

In [4]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Sanity check

We can do a sanity check by removing the highlighted words and check how the prediction changes.

In [5]:
import re

doc2 = re.sub(r"(recall|kidney|stones|medication|pain|tech)", "", doc, flags=re.I)
print_prediction(doc2)

0.067 alt.atheism
0.150 comp.graphics
0.354 sci.med
0.428 soc.religion.christian


The predicted probabilities is indeed lower now.

In [6]:
print(te.samples_[0])

As    my   kidney ,  isn' any
  can        .

Either they ,     be    ,   
to   .

   ,  - tech  to mention  ' had kidney
 and ,     .


In [7]:
# By default, TextExplainer generates 5000 distorted texts.
len(te.samples_)

5000

In [8]:
te.vec_, te.clf_

(CountVectorizer(ngram_range=(1, 2), token_pattern='(?u)\\b\\w+\\b'),
 SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',
               random_state=RandomState(MT19937) at 0x13144B040))

In [9]:
te.metrics_

{'mean_KL_divergence': 0.019433059538127587, 'score': 0.9850240438373423}